In [10]:
from torch import Tensor, tensor, empty, cat, arange, int64
from torch_sparse import SparseTensor
from typing import Optional

class graphTorch():
    
    def __init__(self):
        self.node_identifiers = empty(0, dtype = int64)
        self.node_data = {}
        self.edge_data = {}
        self._counter = 0
        
    def get_nodes_num(self):
        return self.node_identifiers.shape[0]
    
    def get_node_IDs(self):
        return self.node_identifiers
    
    def get_relative_node_indices(self, node_ids: Tensor):
        identifiers = self.node_identifiers.unsqueeze(dim = 1)
        mask = identifiers == node_ids.unsqueeze(dim = 0)
        relative_indices = torch.arange(self.node_identifiers.shape[0]).unsqueeze(dim = 1) * mask
        return relative_indices.sum(dim = 0)    

    def get_node_data(self, node_type: str):
        assert (node_type in self.node_data), "the given key does not exist"
        return self.node_data[node_type]
    
    def get_edges(self, edge_type: str):
        assert (edge_type in self.edge_data), "the given key does not exist"
        adj = self.edge_data[edge_type]
        U = adj.storage.row()
        V = adj.storage.col()
        values = adj.storage.value()
        return U, V, values
        
                
    def add_nodes(self, nodes_data: dict):
        new_node_identifiers = arange(self._counter, self._counter + len(nodes_data[next(iter(nodes_data))]))
        self.node_identifiers = cat((self.node_identifiers, new_node_identifiers), dim = 0)
        self._counter += len(new_node_identifiers)
        for node_data_type, data in nodes_data.items():
            if node_data_type in self.node_data:
                self.node_data[node_data_type] = cat((self.node_data[node_data_type], data), dim=0)
            else:
                if self.node_identifiers.shape[0] != 0:
                    assert (data.shape[0] == self.node_identifiers.shape[0]), "number of nodes must be equal"
                self.node_data[node_data_type] = data  
    
    def add_edges(self, edge_type: str, U: Tensor, V: Tensor, directed: bool, edge_weights: Optional[Tensor] = None):
        if directed == False:
            a = cat((U,V), dim = 0)
            b = cat((V,U), dim = 0)
            U,V = a, b
        N = self.get_nodes_num()
        if edge_type not in self.edge_data:
            self.edge_data[edge_type] = SparseTensor(row=U, col=V, sparse_sizes=(N, N), value = edge_weights).coalesce()
        else:
            r,c,v = self.get_edges(edge_type)
            if self.edge_data[edge_type].storage.value() != None:
                raise "the given edge_type must be weighted"
            new_row = cat((r, U), dim = 0)
            new_col = cat((c, V), dim = 0)
            if edge_weights is not None:
                v = cat((v, edge_weights), dim = 0)
            self.edge_data[edge_type] = SparseTensor(row=new_row, 
                                                     col=new_col, 
                                                     sparse_sizes=(N, N), 
                                                     value = v).coalesce()
            
    def delete_edges(self, edge_type: str, U: Tensor, V: Tensor):
        assert (edge_type in self.edge_data), "the given key does not exist"
        N = self.get_nodes_num()
        r,c,v = self.get_edges(edge_type)
        E = cat((r.unsqueeze(0), c.unsqueeze(0)),dim=0).T
        Erem = cat((U.unsqueeze(0), V.unsqueeze(0)), dim=0).T
        mask = E.unsqueeze(1) == Erem
        mask = mask.all(-1)
        non_repeat_mask = ~mask.any(-1)
        if v is not None:
            v = v[non_repeat_mask]
        self.edge_data[edge_type] = SparseTensor(row=r[non_repeat_mask], 
                                                 col=c[non_repeat_mask], 
                                                 sparse_sizes=(N, N), 
                                                 value = v).coalesce()
    
    def delete_nodes(self, node_identifiers: Optional[Tensor] = None, relative_node_indices: Optional[Tensor] = None):
        assert ((node_identifiers != None) ^ (relative_node_indices != None)), "Either node_identifiers, or relative_node_indices have to be provided"
        if node_identifiers != None:
            relative_node_indices = self.get_relative_node_indices(node_identifiers)
        relative_node_indices = cat((relative_node_indices.unsqueeze(0),relative_node_indices.unsqueeze(0)), dim = 0).T
        N = self.get_nodes_num()
        for key in self.edge_data:
            r,c,v = self.get_edges(key)
            E = cat((r.unsqueeze(0), c.unsqueeze(0)), dim=0).T
            mask = (E.unsqueeze(1) == relative_node_indices).any(-1).any(-1)
            if v is not None:
                v = v[mask]
            self.edge_data[key] = SparseTensor(row=r[mask], col=c[mask], sparse_sizes=(N, N), value = v).coalesce()
        nodes_with_new_relative_indices = arange(self.node_identifiers.shape[0] - relative_node_indices.shape[0], N)
        for ind, elm in enumerate(relative_node_indices[:,0]):
            self.node_identifiers[elm] = self.node_identifiers[-1]
            self.node_identifiers = self.node_identifiers[:-1]
            for k in self.node_data:
                self.node_data[k][elm] = self.node_data[k][-1]
                self.node_data[k] = self.node_data[k][:-1]
            for k in self.edge_data:
                r,c,v = self.get_edges(k)
                Umask = r == nodes_with_new_relative_indices[-(ind+1)]
                Vmask = c == nodes_with_new_relative_indices[-(ind+1)]
                r[Umask] = elm
                c[Vmask] = elm
                self.edge_data[k] = SparseTensor(row=r, 
                                                 col=c, 
                                                 sparse_sizes=(N, N), 
                                                 value = v).coalesce()

        

dg = graphTorch()

from torch import rand
dg.add_nodes({"w": rand(5,6,6), "s": rand(5,4,4)})
#dg.add_nodes({"w": torch.rand(2,6,6), "s": torch.rand(2,4,4)})
#dg.add_edges(edge_type = "w", U = tensor([1,2,1,4,2,0]), V = tensor([2,2,2,4,3,1]), values = tensor([2,3,9,10,0,2]), directed = True)
dg.add_edges(edge_type = "w", U = tensor([1,2,1,4,2,0]), V = tensor([2,2,2,4,3,1]), directed = False)
#dg.delete_edges("w", U = tensor([4,1,2]), V = tensor([4,2,2]))
#print(dg.edge_data["w"].U, dg.edge_data["w"].V, dg.edge_data["w"].values)

#dg.delete_nodes(node_identifiers = tensor([2,3]))
#dg.get_relative_node_indices(tensor([1,2,3]))
dg.edge_data['w']

SparseTensor(row=tensor([0, 1, 1, 2, 2, 2, 3, 4]),
             col=tensor([1, 0, 2, 1, 2, 3, 2, 4]),
             size=(5, 5), nnz=8, density=32.00%)

In [18]:
from torch_sparse import SparseTensor, fill_diag, sum, mul
from torch import nn.Module
from typing import Optional

def gcn_norm(adj: SparseTensor, add_self_loops: bool = True):
    if adj.has_value() == False:
        adj = adj.fill_value(1)
    if add_self_loops:
        A_tilde = fill_diag(adj_t, 1)
    D_tilde = sum(A_tilde, dim = 1)
    D_tilde_inv_sqrt = D_tilde.pow_(-0.5)
    D_tilde_inv_sqrt.masked_fill_(D_tilde_inv_sqrt == float('inf'), 0)
    adj_hat = mul(mul(A_tilde, deg_inv_sqrt.unsqueeze(0)), deg_inv_sqrt.unsqueeze(0))
    return adj_hat

class GCN_layer(nn.Module):
    
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 add_self_loops: bool = True 
                ):
        super(GCN_layer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.add_self_loops = add_self_loops
        
    

tensor([1, 0, 2, 1, 2, 3, 2, 4])

In [217]:
dg.edge_data['w'].U

tensor([0, 1, 1, 2, 2, 2, 3, 4])

In [4]:
N = dg.node_identifiers.shape[0]
adj = SparseTensor(row = dg.edge_data['w'].U, 
                   col = dg.edge_data['w'].V, 
                   sparse_sizes = (N, N)
                  )
adj.has_value()

False

In [6]:
adj = adj.fill_value(1.)

In [13]:
adj

SparseTensor(row=tensor([0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 4]),
             col=tensor([0, 1, 0, 1, 2, 1, 2, 3, 2, 3, 4]),
             val=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
             size=(5, 5), nnz=11, density=44.00%)

In [22]:
sum(adj, dim = 1).shape

torch.Size([5])

In [21]:
sum(adj, dim = 1).view(1,-1).shape

torch.Size([1, 5])